In [ ]:
from math import floor

def max_pool(height, kernel_size, stride=None, padding=0, dilation=1):
    if stride is None:
        stride = kernel_size
    return floor((height + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)

def conv(height, kernel_size, stride=1, padding=0, dilation=1):
    return floor((height + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)

In [ ]:
sizes = [conv(400, 3)]
for i in range(0, 8, 2):
    sizes.append(max_pool(sizes[i], 2, stride=2))
    sizes.append(conv(sizes[i+1], 3))
sizes.append(max_pool(sizes[8], 2, stride=2))
sizes

In [ ]:
[s ** 2 for s in sizes]

In [ ]:
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [ ]:
data_dir = os.path.join("fich_database", "inputs")
dataset = datasets.ImageFolder(data_dir, transforms.ToTensor())

In [ ]:
sizes = list()
for image, label in dataset:
    s = transforms.functional.get_image_size(image)
    sizes.append(s)

In [ ]:
import statistics as stat

In [ ]:
def stats(sizes, f):
    f0 = f([s[0] for s in sizes])
    f1 = f([s[1] for s in sizes])
    return f0, f1

In [ ]:
stats(sizes, stat.stdev)

In [ ]:
stats(sizes, stat.mean)

In [ ]:
stats(sizes, min)

In [ ]:
stats(sizes, max)

In [ ]:
import pickle

In [ ]:
with open('image_sizes.pkl', 'wb') as file:
    pickle.dump(sizes, file)

In [ ]:
with open('image_sizes.pkl', 'rb') as file:
    loaded_object = pickle.load(file)

In [ ]:
stats(loaded_object, max)

In [ ]:
import torch

In [ ]:
torch.Tensor([2, 1]) * 2

In [ ]:
any([0, 1, 1])